In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/ForecastFrontiers/Electricity/data

[Errno 2] No such file or directory: 'drive/MyDrive/ForecastFrontiers/Electricity/data'
/content


# Data Preprocessing

## Read Data

**Dataset Overview:**

Dataset has **no missing values**.

**Data time horizon**: 2011-01-01 to 2015-01-01.

Values are in **kW of each 15 min**. To convert values in kWh values must be divided by 4.

First column present **date and time** as a string with the following format 'yyyy-mm-dd hh:mm:ss'. For other 370 columns, each column represent **one client**. Some clients were created after 2011. In these cases consumption were considered zero.

All time labels report to Portuguese hour. However all days present **96 measures** (24*4).

Every year in March time change day (which has only 23 hours) the **values between 1:00 am and 2:00 am are zero for all points**. Every year in October time change day (which has 25 hours) the **values between 1:00 am and 2:00 am aggregate** the consumption of two hours.



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore", category=UserWarning)

In [ ]:
df = pd.read_csv("LD2011_2014.txt", sep=';', low_memory=False, decimal=',', parse_dates=[0], index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'LD2011_2014.txt'

In [ ]:
df.head()

## Check Data

In [ ]:
# check data type
df.info()

In [ ]:
# check columns
df.columns

In [ ]:
df.describe()

In [ ]:
# check time difference
df.index.to_series().diff().value_counts()

Daylight saving time does not require adjustment as the dataset maintains a consistent 15-minute interval, ensuring uniform time indexing.

## Preprocess Data

In [ ]:
# kW to kWh
df /= 4

In [ ]:
# aggregate by day
df_daily = df.resample('D').sum()

In [ ]:
df_daily.to_csv('daily_usage_wide.csv')

In [ ]:
# convert to long table format for modeling
daily_usage = df_daily.melt(ignore_index=False, var_name="Account", value_name="Usage").reset_index().rename(columns={'index':'Datetime'})

In [ ]:
daily_usage.info()

In [ ]:
daily_usage.head()

In [ ]:
daily_usage.to_csv('daily_usage_individual.csv')

In [ ]:
daily_overall = daily_usage.groupby('Datetime')['Usage'].sum().reset_index()

In [ ]:
daily_overall.info()

In [ ]:
daily_overall.head()

In [ ]:
daily_overall.to_csv('daily_usage_overall.csv')

In [ ]:
# aggregate by week
df_weekly = df.resample('W').sum()
df_weekly = df_weekly.iloc[:-1]

In [ ]:
df_weekly.to_csv('weekly_usage_wide.csv')

In [ ]:
# convert to long table format for modeling
weekly_usage = df_weekly.melt(ignore_index=False, var_name="Account", value_name="Usage").reset_index().rename(columns={'index':'Datetime'})

In [ ]:
weekly_usage.info()

In [ ]:
weekly_usage.head()

In [ ]:
weekly_usage.to_csv('weekly_usage_individual.csv')

In [ ]:
weekly_overall = weekly_usage.groupby('Datetime')['Usage'].sum().reset_index()

In [ ]:
weekly_overall.info()

In [ ]:
weekly_overall.head()

In [ ]:
weekly_overall.to_csv('weekly_usage_overall.csv')

# Exploratory Data Analysis

In [ ]:
# individual account usage
weekly_usage = pd.read_csv("weekly_usage_individual.csv")
weekly_usage = weekly_usage.drop(columns=['Unnamed: 0'], errors='ignore')
weekly_usage['Datetime'] = pd.to_datetime(weekly_usage['Datetime'])
# overall usage
weekly_overall = pd.read_csv("weekly_usage_overall.csv")
weekly_overall = weekly_overall.drop(columns=['Unnamed: 0'], errors='ignore')
weekly_overall['Datetime'] = pd.to_datetime(weekly_overall['Datetime'])

FileNotFoundError: [Errno 2] No such file or directory: 'weekly_usage_individual.csv'

In [ ]:
# weekly_usage.info()

In [ ]:
# weekly_overall.info()

## Overall Trend

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(weekly_overall['Datetime'], weekly_overall['Usage'], marker='o', linestyle='-')
plt.xlabel("Date")
plt.ylabel("Total Weekly Usage (kWh)")
plt.title("Weekly Overall Electricity Usage")
plt.grid()
plt.show()

## Moving Average

In [ ]:
moving_avg = weekly_overall.copy()
moving_avg['moving_avg'] = moving_avg['Usage'].rolling(window=4, center=True).mean()

plt.figure(figsize=(12, 5))
plt.plot(moving_avg.index, moving_avg['Usage'], label="Actual", alpha=0.5)
plt.plot(moving_avg.index, moving_avg['moving_avg'], label="4-week Moving Average", color='red')
plt.xlabel("Date")
plt.ylabel("Total Weekly Usage (kWh)")
plt.title("Moving Average of Weekly Overall Usage")
plt.legend()
plt.grid()
plt.show()

## Autocorrelation

In [ ]:
from matplotlib.ticker import AutoMinorLocator

plt.figure(figsize=(10, 5))
plt.acorr(weekly_overall['Usage'].values, maxlags=None)
plt.xlabel(r"$\tau$")
plt.ylabel("Autocorrelation")
plt.legend([r"$R_X(\tau)$"])
plt.title("Autocorrelation of Weekly Electricity Usage")
plt.grid()
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, ax = plt.subplots(2, 1, figsize=(12, 8))
plot_acf(weekly_overall['Usage'], lags=200, ax=ax[0])
plot_pacf(weekly_overall['Usage'], lags=104, ax=ax[1])
plt.show()

## PSD

In [ ]:
from scipy import signal

freqs, psd = signal.welch(weekly_overall['Usage'].values, fs=1, nperseg = 210)

plt.figure(figsize=(10, 5))
plt.plot(freqs, psd, marker='o', linestyle='-')
plt.xlabel("Frequency")
plt.ylabel("Power Spectral Density")
plt.legend(['PSD'])
plt.title("Power Spectral Density (PSD) of Weekly Electricity Usage")
plt.grid()
plt.show()

## Seasonal Decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(weekly_overall['Usage'], model='additive', period=52)
result.plot()
plt.show()

## Check Outliers

In [ ]:
threshold = weekly_overall['Usage'].mean() + 3 * weekly_overall['Usage'].std()
outliers = weekly_overall[weekly_overall['Usage'] > threshold]

plt.figure(figsize=(12, 5))
plt.plot(weekly_overall.index, weekly_overall['Usage'], label="Usage", alpha=0.5)
plt.scatter(outliers.index, outliers['Usage'], color='red', label="Outliers")
plt.xlabel("Date")
plt.ylabel("Total Weekly Usage (kWh)")
plt.title("Anomaly Detection in Weekly Usage")
plt.legend()
plt.grid()
plt.show()

print(outliers)

# Model: SARIMA

1. **d = 1** (Non-seasonal Differencing)

The original data shows a long-term increasing trend. ACF decreases gradually, indicating the need for differencing.

2. **D = 1** (Seasonal Differencing)

ACF at lag=52 remains significant, confirming seasonal influence.

3. **p = 1** (Non-seasonal Autoregressive Order)

PACF cuts off at lag=1, suggesting an AR(1) process.

4. **q = 1** (Non-seasonal Moving Average Order)

ACF cuts off at lag=1, indicating an MA(1) process.

5. **P = 1** (Seasonal Autoregressive Order)

ACF shows a peak at lag=52, confirming the need for a seasonal AR term.

6. **Q = 1** (Seasonal Moving Average Order)

PACF does not show a strong peak at lag=52, but selecting Q=1 ensures model robustness.

7. **s = 52** (Seasonal Periodicity)

The ACF plot shows a strong peak at lag=52, indicating an annual seasonal cycle. The seasonal decomposition confirms a repeating pattern every year.

In [ ]:
# individual account usage
weekly_usage = pd.read_csv("weekly_usage_individual.csv")
weekly_usage = weekly_usage.drop(columns=['Unnamed: 0'], errors='ignore')
weekly_usage['Datetime'] = pd.to_datetime(weekly_usage['Datetime'])
# overall usage
weekly_overall = pd.read_csv("weekly_usage_overall.csv")
weekly_overall = weekly_overall.drop(columns=['Unnamed: 0'], errors='ignore')
weekly_overall['Datetime'] = pd.to_datetime(weekly_overall['Datetime'])

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore", category=UserWarning)

from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# train test split for overall usage
train_size = int(len(weekly_overall) * 0.6)
val_size = int(len(weekly_overall) * 0.2)

train = weekly_overall.iloc[:train_size]
val = weekly_overall.iloc[train_size:train_size + val_size]
test = weekly_overall.iloc[train_size + val_size:]

## Plot-based Params

### Weekly Overall

#### SARIMA (1,1,1)x(1,1,1,52)

In [ ]:
# train SARIMA model
sarima_model = SARIMAX(train['Usage'],
                        order=(1,1,1),
                        seasonal_order=(1,1,1,52))

sarima_result = sarima_model.fit()

val_pred = sarima_result.predict(start=val.index[0], end=val.index[-1])
test_pred = sarima_result.predict(start=test.index[0], end=test.index[-1])

val_pred.index = val.index
test_pred.index = test.index
mape_val = mape(val['Usage'], val_pred)
mape_test = mape(test['Usage'], test_pred)

print(f"Overall Validation MAPE: {mape_val:.2f}%")
print(f"Overall Test MAPE: {mape_test:.2f}%")

In [ ]:
plt.figure(figsize=(12,6))

plt.plot(train.index, train['Usage'], label="Train (Overall)", color='blue')
plt.plot(val.index, val['Usage'], label="Validation (Overall)", color='green')
plt.plot(test.index, test['Usage'], label="Test (Overall)", color='black')

plt.plot(val.index, val_pred, label="SARIMA Validation Forecast (Overall)", linestyle='dashed', color='orange')
plt.plot(test.index, test_pred, label="SARIMA Test Forecast (Overall)", linestyle='dashed', color='red')

plt.xlabel("Date")
plt.ylabel("Total Weekly Usage (kWh)")
plt.title("SARIMA Model - Overall Actual vs Forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# split test set into 3 equal regions
test_split = np.array_split(test.index, 3)

mape_results = []
for i, period in enumerate(test_split):
    mape_period = mape(test.loc[period, 'Usage'], test_pred.loc[period])
    mape_results.append(mape_period)
    print(f"Overall MAPE for Test Period {i+1}: {mape_period:.2f}%")

error_df = []
for i, period in enumerate(test_split):
    period_error = test.loc[period, 'Usage'] - test_pred.loc[period]
    error_df.append(pd.DataFrame({'Error': period_error, 'Test Period': f"Overall Period {i+1}"}))

error_df = pd.concat(error_df)

plt.figure(figsize=(8,5))
sns.boxplot(x="Test Period", y="Error", data=error_df)
plt.title("Overall Error Distribution Across Test Periods")
plt.xlabel("Test Period (Overall)")
plt.ylabel("Prediction Error")
plt.grid()
plt.show()

## Search-based Params

### Weekly Individual Account

In [ ]:
!pip install pmdarima

In [ ]:
from pmdarima import auto_arima

auto_model = auto_arima(weekly_overall['Usage'], seasonal=True, m=52, trace=False)
print(auto_model.summary())

p, d, q = auto_model.order
P, D, Q, s = auto_model.seasonal_order
print("Method: auto_arima")
print(f"Best p = {p}, d = {d}, q = {q}")
print(f"Best P = {P}, D = {D}, Q = {Q}, s = {s}")

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf

def check_stationarity(timeseries):
    adf_result = adfuller(timeseries.dropna())
    kpss_result = kpss(timeseries.dropna(), regression='c', nlags="auto")

    adf_stationary = adf_result[1] < 0.05  # ADF p-value < 0.05 -> Stationary
    kpss_stationary = kpss_result[1] > 0.05  # KPSS p-value > 0.05 -> Stationary

    return adf_stationary and kpss_stationary

def find_optimal_d_D(timeseries, max_d=2, max_D=2):
    # find d
    d = 0
    while d < max_d:
        print(f"Testing d={d}...")
        if check_stationarity(timeseries):
            print(f"Data is stationary at d={d}")
            break
        else:
            print(f"Data is not stationary at d={d}, applying differencing...")
            timeseries = timeseries.diff().dropna()
            d += 1
    else:
        print("Warning: Data is still not stationary after max_d differencing!")

    # find D
    D = 0
    while D < max_D:
        print(f"Testing D={D}...")
        # plot_acf(timeseries, lags=104)
        # plt.title(f"ACF Plot for D={D}")
        # plt.show()
        if abs(timeseries.autocorr(lag=52)) < 0.3:  # ACF(lag=52) < 0.3 -> Non-Seasonality
            print(f"Data is seasonally stationary at D={D}")
            break
        else:
            print(f"Data still has strong seasonality at D={D}, applying seasonal differencing...")
            timeseries = timeseries.diff(52).dropna()
            D += 1
    else:
        print("Warning: Data still shows seasonality after max_D differencing!")

    return d, D

d_opt, D_opt = find_optimal_d_D(weekly_overall['Usage'])
print(f"Optimal values: d={d_opt}, D={D_opt}")

In [ ]:
# SARIMA(D=0)
model_D0 = SARIMAX(weekly_overall['Usage'], order=(1,1,1), seasonal_order=(1,0,1,52))
result_D0 = model_D0.fit()
test_pred_D0 = result_D0.predict(start=test.index[0], end=test.index[-1])
mape_D0 = mape(test['Usage'], test_pred_D0)

# SARIMA(D=1)
model_D1 = SARIMAX(weekly_overall['Usage'], order=(1,1,1), seasonal_order=(1,1,1,52))
result_D1 = model_D1.fit()
test_pred_D1 = result_D1.predict(start=test.index[0], end=test.index[-1])
mape_D1 = mape(test['Usage'], test_pred_D1)

# SARIMA(D=2)
model_D2 = SARIMAX(weekly_overall['Usage'], order=(1,1,1), seasonal_order=(1,2,1,52))
result_D2 = model_D2.fit()
test_pred_D2 = result_D2.predict(start=test.index[0], end=test.index[-1])
mape_D2 = mape(test['Usage'], test_pred_D2)

print(f"MAPE with D=0: {mape_D0:.2f}%")
print(f"MAPE with D=1: {mape_D1:.2f}%")
print(f"MAPE with D=2: {mape_D2:.2f}%")

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from itertools import product

p_values = q_values = [0, 1]
P_values = Q_values = [0, 1]
s = 52
d, D = 1, 1

def grid_search_sarima(d, D):
    best_mape = float("inf")
    best_params = None
    best_model = None

    print(f"\n🔍 Running Grid Search (d={d}, D={D})...")

    for order in product(p_values, [d], q_values):
        for seasonal_order in product(P_values, [D], Q_values):
            try:
                model = SARIMAX(weekly_overall['Usage'], order=order, seasonal_order=seasonal_order + (s,))
                result = model.fit()

                val_pred = result.predict(start=val.index[0], end=val.index[-1])
                mape_val = mape(val['Usage'], val_pred)

                if mape_val < best_mape:
                    best_mape = mape_val
                    best_params = (order, seasonal_order + (s,))
                    best_model = result

            except Exception as e:
                print(f"Error for {order}, {seasonal_order}: {e}")

    print(f"Best Params: {best_params}")
    print(f"Validation MAPE: {best_mape:.2f}%")
    return best_params, best_mape, best_model

best_params, best_mape, best_model = grid_search_sarima(d, D)

test_pred = best_model.predict(start=test.index[0], end=test.index[-1])
mape_test = mape(test['Usage'], test_pred)

print(f"Test MAPE: {mape_test:.2f}%")

In [ ]:
individual_results = {}
individual_accounts = weekly_usage['Account'].unique()

for account in individual_accounts:
    print(f"Applying Best SARIMA for {account}...")

    account_data = weekly_usage[weekly_usage['Account'] == account].set_index('Datetime')['Usage']

    # train test split
    train_size = int(len(account_data) * 0.6)
    val_size = int(len(account_data) * 0.2)

    train = account_data.iloc[:train_size]
    val = account_data.iloc[train_size:train_size + val_size]
    test = account_data.iloc[train_size + val_size:]

    try:
        # train SARIMA model with best params
        model = SARIMAX(train, order=best_params[0], seasonal_order=best_params[1])
        result = model.fit()

        val_pred = result.predict(start=val.index[0], end=val.index[-1])
        test_pred = result.predict(start=test.index[0], end=test.index[-1])

        mape_val = mape(val, val_pred)
        mape_test = mape(test, test_pred)

        individual_results[account] = {
            "Validation MAPE": mape_val,
            "Test MAPE": mape_test,
            "val_pred": val_pred,
            "test_pred": test_pred
        }

        print(f"{account}: Validation MAPE = {mape_val:.2f}%, Test MAPE = {mape_test:.2f}%")

    except Exception as e:
        print(f"Error training SARIMA for {account}: {e}")

print("Applied Best SARIMA to All Accounts!")

In [ ]:
import pickle

# save individual SARIMA results
with open("individual_sarima_results.pkl", "wb") as f:
    pickle.dump(individual_results, f)

print("Individual SARIMA results saved!")

### Weekly Overall

#### SARIMA (0,1,2)x(1,0,0,52)

In [ ]:
# train SARIMA model
sarima_model = SARIMAX(train['Usage'],
                        order=(0,1,0),
                        seasonal_order=(1,0,0,52))

sarima_result = sarima_model.fit()

val_pred = sarima_result.predict(start=val.index[0], end=val.index[-1])
test_pred = sarima_result.predict(start=test.index[0], end=test.index[-1])

val_pred.index = val.index
test_pred.index = test.index
mape_val = mape(val['Usage'], val_pred)
mape_test = mape(test['Usage'], test_pred)

print(f"Overall Validation MAPE: {mape_val:.2f}%")
print(f"Overall Test MAPE: {mape_test:.2f}%")

In [ ]:
plt.figure(figsize=(12,6))

plt.plot(train.index, train['Usage'], label="Train (Overall)", color='blue')
plt.plot(val.index, val['Usage'], label="Validation (Overall)", color='green')
plt.plot(test.index, test['Usage'], label="Test (Overall)", color='black')

plt.plot(val.index, val_pred, label="SARIMA Validation Forecast (Overall)", linestyle='dashed', color='orange')
plt.plot(test.index, test_pred, label="SARIMA Test Forecast (Overall)", linestyle='dashed', color='red')

plt.xlabel("Date")
plt.ylabel("Total Weekly Usage (kWh)")
plt.title("SARIMA Model - Overall Actual vs Forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# split test set into 3 equal regions
test_split = np.array_split(test.index, 3)

mape_results = []
for i, period in enumerate(test_split):
    mape_period = mape(test.loc[period, 'Usage'], test_pred.loc[period])
    mape_results.append(mape_period)
    print(f"Overall MAPE for Test Period {i+1}: {mape_period:.2f}%")

error_df = []
for i, period in enumerate(test_split):
    period_error = test.loc[period, 'Usage'] - test_pred.loc[period]
    error_df.append(pd.DataFrame({'Error': period_error, 'Test Period': f"Overall Period {i+1}"}))

error_df = pd.concat(error_df)

plt.figure(figsize=(8,5))
sns.boxplot(x="Test Period", y="Error", data=error_df)
plt.title("Overall Error Distribution Across Test Periods")
plt.xlabel("Test Period (Overall)")
plt.ylabel("Prediction Error")
plt.grid()
plt.show()

#### SARIMA (0,1,0)x(1,1,1,52)

In [ ]:
# train SARIMA model
sarima_model = SARIMAX(train['Usage'],
                        order=(0,1,0),
                        seasonal_order=(1,1,1,52))

sarima_result = sarima_model.fit()

val_pred = sarima_result.predict(start=val.index[0], end=val.index[-1])
test_pred = sarima_result.predict(start=test.index[0], end=test.index[-1])

val_pred.index = val.index
test_pred.index = test.index
mape_val = mape(val['Usage'], val_pred)
mape_test = mape(test['Usage'], test_pred)

print(f"Overall Validation MAPE: {mape_val:.2f}%")
print(f"Overall Test MAPE: {mape_test:.2f}%")

In [ ]:
plt.figure(figsize=(12,6))

plt.plot(train.index, train['Usage'], label="Train (Overall)", color='blue')
plt.plot(val.index, val['Usage'], label="Validation (Overall)", color='green')
plt.plot(test.index, test['Usage'], label="Test (Overall)", color='black')

plt.plot(val.index, val_pred, label="SARIMA Validation Forecast (Overall)", linestyle='dashed', color='orange')
plt.plot(test.index, test_pred, label="SARIMA Test Forecast (Overall)", linestyle='dashed', color='red')

plt.xlabel("Date")
plt.ylabel("Total Weekly Usage (kWh)")
plt.title("SARIMA Model - Overall Actual vs Forecast")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# split test set into 3 equal regions
test_split = np.array_split(test.index, 3)

mape_results = []
for i, period in enumerate(test_split):
    mape_period = mape(test.loc[period, 'Usage'], test_pred.loc[period])
    mape_results.append(mape_period)
    print(f"Overall MAPE for Test Period {i+1}: {mape_period:.2f}%")

error_df = []
for i, period in enumerate(test_split):
    period_error = test.loc[period, 'Usage'] - test_pred.loc[period]
    error_df.append(pd.DataFrame({'Error': period_error, 'Test Period': f"Overall Period {i+1}"}))

error_df = pd.concat(error_df)

plt.figure(figsize=(8,5))
sns.boxplot(x="Test Period", y="Error", data=error_df)
plt.title("Overall Error Distribution Across Test Periods")
plt.xlabel("Test Period (Overall)")
plt.ylabel("Prediction Error")
plt.grid()
plt.show()